In [1]:
from google.auth import default
credentials, project = default()

In [23]:
%load_ext autoreload
%autoreload 2
import os
import sys
from pathlib import Path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from dotenv import load_dotenv
load_dotenv()

from src.config_loader import config
from src.translation import review_translations_with_anthropic, review_translated_phrases_batch
from src.utils import get_first_n_items
from src.gcs_storage import get_translated_phrases_path, read_from_gcs, upload_to_gcs
config.TARGET_LANGUAGE_NAME

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'Spanish'

# Translation Refinement

## Issues with Google Translate

It can sound stiff / formal and be more related to written text rather than verbal.

## Approach

Use Sonnet 3.5 via the Anthropic API, using a tool to return adjusted translations in JSON format for re-uploading to our phrase translation store (json file)

In [20]:
# get current translated JSON file
translated_file_path =get_translated_phrases_path(collection="LM1000")

phrase_translations = read_from_gcs(bucket_name=config.GCS_PRIVATE_BUCKET, file_path=translated_file_path)

In [24]:
test_dict = get_first_n_items(phrase_translations, 20)

In [22]:
len(phrase_translations)

830

In [27]:
improved_translations = review_translated_phrases_batch(phrase_translations, model = "claude-3-5-sonnet-latest")

Processing batch 1 with 20 phrases...
Processing batch 2 with 20 phrases...
Processing batch 3 with 20 phrases...
Processing batch 4 with 20 phrases...
Processing batch 5 with 20 phrases...
Processing batch 6 with 20 phrases...
Processing batch 7 with 20 phrases...
Processing batch 8 with 20 phrases...
Processing batch 9 with 20 phrases...
Processing batch 10 with 20 phrases...
Processing batch 11 with 20 phrases...
Processing batch 12 with 20 phrases...
Processing batch 13 with 20 phrases...
Processing batch 14 with 20 phrases...
Processing batch 15 with 20 phrases...
Processing batch 16 with 20 phrases...
Processing batch 17 with 20 phrases...
Processing batch 18 with 20 phrases...
Processing batch 19 with 20 phrases...
Processing batch 20 with 20 phrases...
Processing batch 21 with 20 phrases...
Processing batch 22 with 20 phrases...
Processing batch 23 with 20 phrases...
Processing batch 24 with 20 phrases...
Processing batch 25 with 20 phrases...
Processing batch 26 with 20 phrase

In [30]:
len(improved_translations)

830

In [29]:
upload_to_gcs(obj=improved_translations, bucket_name=config.GCS_PRIVATE_BUCKET, file_name=translated_file_path)

'gs://audio-language-trainer-private-content/collections/LM1000/spanish/translations.json'